## Load required libraries

In [52]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
import re
import gensim, spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis 
import altair as alt
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
pyLDAvis.enable_notebook()

## Required Utilities 

### Pre-processing Utilities 

In [2]:
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', \
                   'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need',  \
                   'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    lst_text = text.split()
    ## choose only alphbetical words and filter words less than 3 chars
    lst_text = [token.lower() for token in lst_text if token.isalpha() and len(token) >=3]      
#     text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
#     lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
    
    lst_text = [token for token in lst_text if len(token) >=3]
            
    ## back to string from list
#     text = " ".join(lst_text)
    return lst_text


def ngram_processing(texts):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)

    # Fastest way to get a sentence clubbed as trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)   
    
    bigram_words = [bigram_mod[doc] for doc in texts]
    trigram_words = [trigram_mod[bigram_mod[doc]] for doc in bigram_words]
    return(trigram_words)


def lemmatization(texts, allowed_postags=allowed_postags):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return(texts_out)

### Model Creation Utilities 

In [3]:
def model_creation(corpus, id2word, num_topics, workers=None):
    if not workers:
        lda = LdaModel(corpus, id2word = id2word, num_topics=num_topics,
                              random_state=696, update_every=1,
                              chunksize=10, passes=10, 
                              alpha='symmetric', iterations=100,
                              per_word_topics=True)
    else:
        lda = LdaMulticore(corpus, id2word = id2word, workers = workers, num_topics=num_topics,
                              random_state=696,
                              chunksize=10, passes=10, 
                              alpha='symmetric', iterations=100,
                              per_word_topics=True)
        
    
    return(lda)

def compute_coherence_values(id2word, corpus, texts, workers, limit, start=2, step=2):
    coherence_values = []
    perplexity_values = []
    model_list = []
    num_topics_list = []
    
    for num_topics in tqdm(range(start, limit, step)):
        lda_model = model_creation(corpus, id2word, num_topics, workers)
        perplexity_score = lda_model.log_perplexity(corpus)

        coherence_model = CoherenceModel(model=lda_model, texts=texts,
                                dictionary=id2word,coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        model_list.append(lda_model)
        coherence_values.append(coherence_score)
        perplexity_values.append(perplexity_score)
        num_topics_list.append(num_topics)
    return(num_topics_list, model_list, coherence_values, perplexity_values)
        

## Load Data

In [5]:
# Read articles for 2004 for test purpose
basepath = '/mnt/d/Amit/data-science/MADS/SIADS696/'
filepath = 'datasets/raw_data/'
# filepath = 'scrapy/missing_articles/missing_articles/'
filename = 'articles_2004.jl'
filepath_name = basepath + filepath + filename
filepath_name

'/mnt/d/Amit/data-science/MADS/SIADS696/datasets/raw_data/articles_2004.jl'

In [6]:
articles_df = pd.read_json(filepath_name, lines=True)
articles_df = articles_df.dropna()
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54962 entries, 0 to 56740
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        54962 non-null  int64         
 1   date      54962 non-null  datetime64[ns]
 2   url       54962 non-null  object        
 3   title     54962 non-null  object        
 4   category  54962 non-null  object        
 5   article   54962 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.9+ MB


### Pre-processing Text

In [7]:
%%time
articles_df['clean_text'] = articles_df['article'].apply(lambda text : 
                        utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=stop_words))

articles_df.head()

CPU times: user 38.2 s, sys: 129 ms, total: 38.3 s
Wall time: 38.3 s


,id,date,url,title,category,article,clean_text
0,1,2004-01-01,https://timesofindia.indiatimes.com/business/i...,economy-breaks-8-pc-barrier,"[business, india-business]",NEW DELHI: The feel-good factor got a boost on...,"[new, factor, got, boost, last, day, indian, e..."
1,2,2004-01-01,https://timesofindia.indiatimes.com/world/paki...,jaish-leader-missing-after-attacks,"[world, pakistan]",ISLAMABAD: Some activists of the banned milita...,"[activist, banned, militant, outfit, arrested,..."
2,3,2004-01-01,https://timesofindia.indiatimes.com/world/us/1...,1m-for-jackson-interview-to-cbs,"[world, us]","&lt;div class=""section1""&gt;&lt;div class=""Nor...","[michael, jackson, struck, deal, cbs, paid, ef..."
3,4,2004-01-01,https://timesofindia.indiatimes.com/india/sars...,sars-screening-takes-off-at-igi,"[timesofindia.indiatimes.com, india]",NEW DELHI: With the SARS virus raising its hea...,"[new, sars, virus, raising, head, airport, aut..."
4,5,2004-01-01,https://timesofindia.indiatimes.com/india/indi...,india-to-test-fire-agni-iii-soon,"[timesofindia.indiatimes.com, india]",NEW DELHI: The nuclear-capable â€˜Agni-IIIâ€™ ...,"[new, capable, hitting, strategic, target, dee..."


In [89]:
# choosing number of articles to process for stub testing
n = 1000
sample_df = articles_df.sample(n)
texts = list(sample_df['clean_text'].values)
text_grams = ngram_processing(texts)
data_lemmatized = lemmatization(text_grams, allowed_postags)

In [9]:
%%time
# model creation
texts = data_lemmatized
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(doc) for doc in texts]

CPU times: user 1.63 s, sys: 20.1 ms, total: 1.65 s
Wall time: 1.64 s


In [75]:
%%time
# model creation
# time estimation with LDA single core
# num_topics_list, model_list, coherence_vals, perplexity_vals = compute_coherence_values(id2word, corpus,   \
#                                                                 texts,workers=None, limit=30, start=2, step=3)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [10]:
%%time
# time estimation with multicore LDA
# model creation
num_topics_list, model_list, coherence_vals, perplexity_vals = compute_coherence_values(id2word, corpus,    \
                                                                texts, workers=4, limit=25, start=2, step=3)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [28:59<00:00, 217.44s/it]

CPU times: user 22min 15s, sys: 6min 50s, total: 29min 5s
Wall time: 28min 59s


In [18]:
model_df = pd.DataFrame(list(zip(num_topics_list, coherence_vals, perplexity_vals)), 
                        columns = ['num_topics', 'coherence_score', 'perplexity_score']) 

In [19]:
model_df

,num_topics,coherence_score,perplexity_score
0,2,0.366396,-8.486598
1,5,0.432873,-8.550593
2,8,0.472356,-8.754692
3,11,0.453655,-9.728669
4,14,0.424350,-12.066752
5,17,0.420482,-13.253042
6,20,0.408369,-13.779222
7,23,0.388907,-14.552712


## Visualize Topic Distribution

In [20]:
base_chart = alt.Chart(model_df)
line = base_chart.mark_line().encode(
            x=alt.X('num_topics:N', axis=alt.Axis(title='Number of Topics')),
            y=alt.Y('coherence_score', axis=alt.Axis(title='Coherence Score')),
            tooltip = ['num_topics', 'coherence_score']
)

point = base_chart.mark_point(color='orange', size=50).encode(
            x=alt.X('num_topics:N', axis=alt.Axis(title='Number of Topics')),
            y=alt.Y('coherence_score', axis=alt.Axis(title='Coherence Score')),
            tooltip = ['num_topics', 'coherence_score']
)    


(line + point
).properties(width=500, height=300, title='Number of Topics vs Coherence Score'
).configure_title(fontSize=25
).configure_axis(grid=False, domain=False, 
                 labelFontSize=15,titleFontSize=20)

alt.LayerChart(...)

In [21]:
# numbe of topics 8
optimal_model_cond = np.where(model_df['coherence_score'] == max(model_df['coherence_score'].values))
optimal_model = model_list[optimal_model_cond[0][0]]

In [55]:
model_df

,num_topics,coherence_score,perplexity_score
0,2,0.366396,-8.486598
1,5,0.432873,-8.550593
2,8,0.472356,-8.754692
3,11,0.453655,-9.728669
4,14,0.424350,-12.066752
5,17,0.420482,-13.253042
6,20,0.408369,-13.779222
7,23,0.388907,-14.552712


In [67]:
from gensim.test.utils import datapath
temp_file = datapath("/mnt/d/Amit/data-science/MADS/SIADS696/model")
model_list[2].save(temp_file)

In [87]:
for i, row in enumerate(lda[corpus]):
    row = sorted(ro)

[(0, 0.47821423), (2, 0.50142056)]

In [61]:
lda= LdaModel.load(temp_file)

In [64]:
lda.show_topics()

[(0,
  '0.020*"congress" + 0.014*"election" + 0.014*"bjp" + 0.013*"candidate" + 0.013*"party" + 0.011*"say" + 0.010*"seat" + 0.008*"vote" + 0.007*"nda" + 0.006*"poll"'),
 (1,
  '0.015*"indian" + 0.011*"india" + 0.010*"student" + 0.008*"say" + 0.008*"film" + 0.006*"time" + 0.005*"big" + 0.005*"world" + 0.005*"first" + 0.005*"new"'),
 (2,
  '0.015*"say" + 0.009*"new" + 0.009*"government" + 0.007*"state" + 0.006*"project" + 0.006*"official" + 0.005*"department" + 0.005*"work" + 0.005*"give" + 0.005*"year"'),
 (3,
  '0.025*"police" + 0.019*"case" + 0.017*"court" + 0.013*"say" + 0.011*"arrest" + 0.010*"accuse" + 0.008*"order" + 0.008*"file" + 0.007*"officer" + 0.006*"criminal"'),
 (4,
  '0.025*"say" + 0.018*"minister" + 0.016*"state" + 0.015*"government" + 0.011*"chief" + 0.011*"party" + 0.007*"leader" + 0.007*"new" + 0.006*"issue" + 0.006*"bjp"'),
 (5,
  '0.009*"say" + 0.008*"road" + 0.006*"vehicle" + 0.006*"car" + 0.005*"city" + 0.005*"day" + 0.005*"take" + 0.005*"time" + 0.005*"traffic" 

In [23]:
# Number of Topics = 8
gensimvis.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.026103 -0.140055       1        1  21.568060
2     -0.064096 -0.073324       2        1  20.706343
1     -0.128532  0.170873       3        1  11.253039
5      0.047008  0.177541       4        1  10.285210
0      0.000241 -0.112762       5        1   9.947366
3      0.188281 -0.115484       6        1   9.522416
6      0.190777  0.129967       7        1   8.915347
7     -0.259783 -0.036757       8        1   7.802218, topic_info=               Term         Freq        Total Category  logprob  loglift
130          police  5712.000000  5712.000000  Default  30.0000  30.0000
118        minister  4467.000000  4467.000000  Default  29.0000  29.0000
88         congress  3360.000000  3360.000000  Default  28.0000  28.0000
1329          party  4186.000000  4186.000000  Default  27.0000  27.0000
711         company  1893.000000  1893.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
299            year   539.492361  3160.720717   Topic8  -5.0973   0.7828
470          indian   508.469030  2645.298412   Topic8  -5.1566   0.9016
227          expect   407.107257  1073.991709   Topic8  -5.3789   1.5807
33    international   406.636910  1085.391372   Topic8  -5.3800   1.5690
444            last   400.520396  3055.277233   Topic8  -5.3952   0.5189

[609 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
3038       2  0.995495         abroad
0          7  0.997088          abuse
1262       3  0.992978        academy
2120       2  0.994867       accident
5228       4  0.989931  accommodation
...      ...       ...            ...
2731       3  0.814029          young
2731       5  0.005719          young
2731       7  0.146792          young
3241       4  0.987135      youngster
24920      4  0.990954            zoo

[1479 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 6, 1, 4, 7, 8])

In [24]:
# Number of Topics = 5
gensimvis.prepare(model_list[1], corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.006604 -0.140660       1        1  28.265162
4     -0.110784 -0.102243       2        1  21.991548
3     -0.181321  0.106004       3        1  19.033634
1      0.193562 -0.037039       4        1  17.979713
0      0.091940  0.173939       5        1  12.729943, topic_info=          Term         Freq        Total Category  logprob  loglift
130     police  5824.000000  5824.000000  Default  30.0000  30.0000
1329     party  4172.000000  4172.000000  Default  29.0000  29.0000
118   minister  4170.000000  4170.000000  Default  28.0000  28.0000
88    congress  3402.000000  3402.000000  Default  27.0000  27.0000
83         bjp  3296.000000  3296.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
981       much   328.430783  1171.322627   Topic5  -6.0832   0.7897
265     number   337.956057  1524.941981   Topic5  -6.0546   0.5544
339       find   338.265904  1947.308657   Topic5  -6.0537   0.3108
691       take   343.371930  3974.102306   Topic5  -6.0387  -0.3875
318       come   335.048357  2464.781529   Topic5  -6.0632   0.0656

[393 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3107      1  0.994109   academic
1066      2  0.083458     accuse
1066      3  0.916303     accuse
2085      1  0.998871  admission
77        2  0.997234     advani
...     ...       ...        ...
299       3  0.096793       year
299       4  0.243501       year
299       5  0.063067       year
852       3  0.338756      youth
852       5  0.658093      youth

[809 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])

In [53]:
# Number of Topics = 11
gensimvis.prepare(model_list[3], corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.015142  0.142733       1        1  19.012162
9     -0.015790  0.024906       2        1  16.527822
2      0.142721  0.098783       3        1  12.119495
3     -0.161443  0.111972       4        1  10.304419
10     0.137881  0.038684       5        1   8.563926
0     -0.020639 -0.038360       6        1   7.349654
1      0.063133 -0.179674       7        1   6.190017
7      0.281714 -0.007475       8        1   6.035165
8     -0.152332  0.122087       9        1   5.541176
5     -0.159649 -0.015996      10        1   4.831836
6     -0.100454 -0.297661      11        1   3.524327, topic_info=          Term         Freq         Total Category  logprob  loglift
130     police  6356.000000   6356.000000  Default  30.0000  30.0000
1329     party  4572.000000   4572.000000  Default  29.0000  29.0000
88    congress  3699.000000   3699.000000  Default  28.0000  28.0000
83         bjp  3256.000000   3256.000000  Default  27.0000  27.0000
118   minister  4919.000000   4919.000000  Default  26.0000  26.0000
...        ...          ...           ...      ...      ...      ...
30      health   205.789149    851.060499  Topic11  -5.2664   1.9258
339       find   245.804759   2050.134146  Topic11  -5.0887   1.2244
656       help   201.385866   1664.774118  Topic11  -5.2880   1.2333
61         say   274.201487  18582.278726  Topic11  -4.9794  -0.8706
474        law   175.326228    719.246187  Topic11  -5.4266   1.9339

[766 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
0         11  0.990143      abuse
3107       5  0.995147   academic
2120       5  0.996654   accident
1066       1  0.035649     accuse
1066       4  0.963323     accuse
...      ...       ...        ...
299        9  0.038595       year
299       10  0.003689       year
299       11  0.026960       year
3241      10  0.987874  youngster
24920     11  0.992551        zoo

[1994 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 3, 4, 11, 1, 2, 8, 9, 6, 7])

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

In [31]:
# Number of Topics = 14
gensimvis.prepare(model_list[4], corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.120166 -0.032840       1        1  19.169520
4     -0.177364 -0.029829       2        1  15.289635
12    -0.155260  0.062576       3        1  13.880877
0     -0.114070 -0.019727       4        1   9.205581
3     -0.090921 -0.195482       5        1   7.510778
2     -0.058581  0.005673       6        1   6.212371
13    -0.075886 -0.065694       7        1   5.899463
7      0.010387  0.273789       8        1   5.488633
10    -0.026476  0.015253       9        1   4.859785
1      0.004628  0.213836      10        1   4.540776
8      0.055240 -0.192918      11        1   2.725625
6      0.212274 -0.012925      12        1   2.290546
5      0.206369  0.057237      13        1   1.902207
11     0.329826 -0.078948      14        1   1.024204, topic_info=           Term         Freq        Total Category  logprob  loglift
130      police  7489.000000  7489.000000  Default  30.0000  30.0000
1329      party  5402.000000  5402.000000  Default  29.0000  29.0000
88     congress  4477.000000  4477.000000  Default  28.0000  28.0000
288     student  3099.000000  3099.000000  Default  27.0000  27.0000
83          bjp  3894.000000  3894.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2009   incident   110.177303   862.389688  Topic14  -4.6554   2.5236
1025     engine    63.211883   110.697989  Topic14  -5.2110   4.0209
495   authority    85.128922  1114.002743  Topic14  -4.9133   2.0097
24        drive    73.949475   700.477115  Topic14  -5.0541   2.3329
626    security    65.990070  1092.981324  Topic14  -5.1680   1.7741

[914 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        12  0.993105       abuse
3107      3  0.003610    academic
3107      9  0.992777    academic
1        12  0.985223  acceptable
1838      6  0.020317      access
...     ...       ...         ...
299      12  0.004984        year
299      13  0.009018        year
2731      1  0.989664       young
2731     12  0.008544       young
3241     13  0.990645   youngster

[2513 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 13, 1, 4, 3, 14, 8, 11, 2, 9, 7, 6, 12])

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho